In [10]:
import psycopg2
from dotenv import load_dotenv

load_dotenv()

import os
def check_db_entries():
    connection = psycopg2.connect(
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        database=os.getenv("DB_DATABASE")
    )
    try:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM bot_logs;")
        #cursor.execute(f"INSERT INTO MAILING_HISTORY (DATE) VALUES ('{current_date}');")
        records = cursor.fetchall()
        # for record in records:
        #     print(record)
        connection.commit()
        return records
    except Exception as error:
        print(f"An error occurred: {error}")
    finally:
        cursor.close()
        connection.close()

check_db_entries()


[(1,
  datetime.datetime(2025, 1, 14, 1, 46, 52, 649215),
  'INFO',
  '2025-01-14 01:46:53,111 - Table bot_logs created successfully.'),
 (2,
  datetime.datetime(2025, 1, 14, 1, 46, 52, 669270),
  'INFO',
  '2025-01-14 01:46:53,137 - Creating table USERS...'),
 (3,
  datetime.datetime(2025, 1, 14, 1, 46, 52, 727351),
  'INFO',
  '2025-01-14 01:46:53,195 - Table USERS created successfully.'),
 (4,
  datetime.datetime(2025, 1, 14, 1, 46, 52, 772327),
  'INFO',
  '2025-01-14 01:46:53,238 - Creating table MAILING_HISTORY...'),
 (5,
  datetime.datetime(2025, 1, 14, 1, 46, 52, 828712),
  'INFO',
  '2025-01-14 01:46:53,296 - Table MAILING_HISTORY created successfully.'),
 (6,
  datetime.datetime(2025, 1, 14, 1, 46, 52, 873158),
  'INFO',
  '2025-01-14 01:46:53,339 - Database connection closed after table creation.'),
 (7,
  datetime.datetime(2025, 1, 14, 1, 47, 18, 412229),
  'INFO',
  '2025-01-14 01:47:18,879 - Adding user 547495974 to USERS table with default display time: 00:00...'),
 (8,


In [5]:
from db_interaction import *

create_tables_DB()